# Kaggle Machine Learning Project
## Project: Titanic - Machine Learning from Disaster 

The problem to be solved is to predict the passengers that survive the Titanic. The goal is to use passenger characteristics and location to determine if a passenger is likely to die or survive. Based off the Kaggle prompt and the expected typed of output, this problem is a supervised classification problem. A potential way to solve this problem is to use a neural network classifier. 

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.